## Libs

In [1]:
import matplotlib as matpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

# Using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow import set_random_seed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

# advanced plotting
import seaborn as sns
plt.style.use('seaborn-darkgrid')
%matplotlib inline

Using TensorFlow backend.


## Data

In [13]:
%store -r data_EEM

In [14]:
mm = MinMaxScaler(feature_range = (0,1))
dataset = mm.fit_transform(data_EEM)

In [15]:
split = 0.6

train_size = int(len(dataset) * split)
#validation
test_size = len(dataset) - train_size

train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

print("training, test set: " + str((len(train), len(test))))

training, test set: (313, 209)


In [16]:
def input_dataset(dataset, window):
    data_X, data_y = [], []
    for i in range(len(dataset) - window - 1):
        a = dataset[i:(i + window), 0]
        data_X.append(a)
        data_y.append(dataset[i + window, 0])
    return(np.array(data_X), np.array(data_y))


In [17]:
# New testing and training sets for rolling forecast.
window = 1
train_X, train_Y = input_dataset(train, window)
test_X, test_Y = input_dataset(test, window)
print("Original train shape:")
print(train_X.shape)

# Reshape input data to match Keras format.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New train shape:")
print(train_X.shape)

Original train shape:
(311, 1)
New train shape:
(311, 1, 1)


## 1 Epoch

In [7]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = 'mean_squared_error', 
                  optimizer = 'adam')
    earlyStop=EarlyStopping(monitor='val_loss',verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

W0919 00:37:35.130961 4634600896 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 00:37:35.134749 4634600896 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 00:37:35.142807 4634600896 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 00:37:35.632453 4634600896 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0919 00:37:36.157521 4634600896 deprecation.py:323] From //anaconda3/li

Epoch 1/1
311/311 [==============================] - 9s 28ms/step - loss: 0.0065


In [8]:
def prediction_score(model, X, Y):
    # Make predictions using input data
    pred = mm.inverse_transform(model.predict(X))
    # Show Y on original scale
    original_data = mm.inverse_transform([Y])
    # RMSE.
    score = math.sqrt(mean_squared_error(original_data[0], pred[:, 0]))
    return(score, pred)

In [9]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)

print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 3.44 RMSE


## 1000 Epochs

In [10]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/1000
311/311 [==============================] - 9s 29ms/step - loss: 0.0065
Epoch 2/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0046
Epoch 3/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0033
Epoch 4/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0026
Epoch 5/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0022
Epoch 6/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0021
Epoch 7/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0020
Epoch 8/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0020
Epoch 9/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0020
Epoch 10/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0020
Epoch 11/1000
311/311 [==============================] - 4s 14ms/step - loss: 0.0020
Epoch 12/1000
311/311 [==============================] - 4s 12ms/step - lo

Epoch 98/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 99/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 100/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 101/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 102/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 103/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 104/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 105/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 106/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 107/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 108/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 109/1000
311/311 [==============================] - 3s 8ms/st

311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 195/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 196/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 197/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 198/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 199/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 200/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 201/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 202/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 203/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 204/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 205/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.

311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 291/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 292/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 293/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 294/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 295/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 296/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 297/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 298/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 299/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 300/1000
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 301/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.

311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 387/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 388/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 389/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 390/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 391/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 392/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 393/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 394/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 395/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 396/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 397/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.

...

311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 579/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 580/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 581/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 582/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 583/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 584/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 585/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 586/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 587/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 588/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 589/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.

311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 675/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 676/1000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 677/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 678/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 679/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 680/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 681/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 682/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 683/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 684/1000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 685/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.

311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 771/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 772/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 773/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 774/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 775/1000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 776/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 777/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 778/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 779/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 780/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 781/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.

311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 867/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 868/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 869/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 870/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 871/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 872/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 873/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 874/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 875/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 876/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 877/1000
311/311 [==============================] - 2s 8ms/step - loss: 0.

311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 963/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 964/1000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 965/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 966/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 967/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 968/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 969/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 970/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 971/1000
311/311 [==============================] - 3s 10ms/step - loss: 0.0016
Epoch 972/1000
311/311 [==============================] - 3s 10ms/step - loss: 0.0016
Epoch 973/1000
311/311 [==============================] - 3s 10ms/step - loss:

In [12]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)

print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 1.78 RMSE


## 2000 Epochs

In [18]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/2000
311/311 [==============================] - 7s 23ms/step - loss: 0.0064
Epoch 2/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0046
Epoch 3/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0033
Epoch 4/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0026
Epoch 5/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0022
Epoch 6/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0021
Epoch 7/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 8/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 9/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 10/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 11/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0020
Epoch 12/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0020


311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 99/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 100/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 101/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 102/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 103/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 104/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 105/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 106/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0018
Epoch 107/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 108/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 109/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0

311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 195/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 196/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0018
Epoch 197/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 198/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 199/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 200/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0018
Epoch 201/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 202/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 203/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 204/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 205/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.

311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 291/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 292/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 293/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 294/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 295/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 296/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 297/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 298/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 299/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 300/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 301/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.

311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 387/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 388/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 389/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 390/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 391/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 392/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 393/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 394/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 395/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 396/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 397/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.

...

311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1533/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1534/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1535/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1536/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1537/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1538/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1539/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1540/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1541/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1542/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1543/2000
311/311 [==============================] - 3s 9ms/step

311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1628/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1629/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1630/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1631/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1632/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1633/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 1634/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 1635/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 1636/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0016
Epoch 1637/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1638/2000
311/311 [==============================] - 3s 8ms/step

311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1723/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1724/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1725/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1726/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1727/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1728/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1729/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1730/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1731/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1732/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1733/2000
311/311 [==============================] - 3s 9ms/step

311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1818/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1819/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1820/2000
311/311 [==============================] - 3s 10ms/step - loss: 0.0015
Epoch 1821/2000
311/311 [==============================] - 3s 10ms/step - loss: 0.0016
Epoch 1822/2000
311/311 [==============================] - 3s 10ms/step - loss: 0.0015
Epoch 1823/2000
311/311 [==============================] - 3s 10ms/step - loss: 0.0016
Epoch 1824/2000
311/311 [==============================] - 3s 10ms/step - loss: 0.0015
Epoch 1825/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0016
Epoch 1826/2000
311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 1827/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1828/2000
311/311 [==============================] - 3s 9ms

311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1913/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1914/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1915/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1916/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1917/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0016
Epoch 1918/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1919/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1920/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1921/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1922/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 1923/2000
311/311 [==============================] - 3s 8ms/step

In [19]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 1.55 RMSE


## 2500 Epochs

In [20]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2500, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/2500
311/311 [==============================] - 6s 21ms/step - loss: 0.0061
Epoch 2/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0044
Epoch 3/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0032
Epoch 4/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0025
Epoch 5/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0022
Epoch 6/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0021
Epoch 7/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0020
Epoch 8/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 9/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 10/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 11/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 12/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0020


311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 99/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 100/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 101/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 102/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 103/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 104/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 105/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 106/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0018
Epoch 107/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 108/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0018
Epoch 109/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0

311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 195/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 196/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 197/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 198/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 199/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 200/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 201/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 202/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 203/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 204/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 205/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.

311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 291/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 292/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 293/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 294/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 295/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 296/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 297/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 298/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 299/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 300/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0017
Epoch 301/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.

311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 387/2500
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 388/2500
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 389/2500
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 390/2500
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 391/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 392/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 393/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 394/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0017
Epoch 395/2500
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 396/2500
311/311 [==============================] - 2s 8ms/step - loss: 0.0017
Epoch 397/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.

...

311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2103/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2104/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2105/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2106/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2107/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2108/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2109/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2110/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2111/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2112/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2113/2500
311/311 [==============================] - 3s 8ms/step

311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2198/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2199/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2200/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2201/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2202/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2203/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2204/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2205/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2206/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2207/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2208/2500
311/311 [==============================] - 3s 8ms/step

311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 2293/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 2294/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2295/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 2296/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 2297/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 2298/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 2299/2500
311/311 [==============================] - 3s 9ms/step - loss: 0.0015
Epoch 2300/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2301/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2302/2500
311/311 [==============================] - 3s 8ms/step - loss: 0.0015
Epoch 2303/2500
311/311 [==============================] - 3s 8ms/step

311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2388/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2389/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2390/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2391/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2392/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2393/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2394/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2395/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2396/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2397/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2398/2500
311/311 [==============================] - 2s 6ms/step

311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2483/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2484/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2485/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2486/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2487/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2488/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2489/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2490/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2491/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2492/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0015
Epoch 2493/2500
311/311 [==============================] - 2s 6ms/step

In [21]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 1.67 RMSE
